# Assignment 2, 8-Puzzle Programming - A Star

### Your homework must be implemented in this Notebook file. 
### You can add as many cells as you want. However, you are not allowed to touch the code below the line "=============".
### You need to implement the two searching functions and the print result functions.
### For the searching functions, feel free to customize the return data types and parameter lists as long as the function name is as required.


In [99]:
import heapq
import time

def format_state(state):
    return '\n'.join([' '.join(str(x) for x in row) for row in state])

def print_sol(sol, puzzle_num, algorithm):
    print(f"\nSolution of the {puzzle_num} Scenario using {algorithm}:")
    for i, state in enumerate(sol):
        if i > 0:
            print("\nto\n")
        print(format_state(state))
    print("\n" + "="*50)
    

In [100]:
def a_star_misplaced(init_state, goal_state):
    def h_misplaced(state):
        return sum(1 for i in range(3) for j in range(3) 
                  if state[i][j] != 0 and state[i][j] != goal_state[i][j])
    
    heap = [(0 + h_misplaced(init_state), 0, init_state, [])]
    visited = set()
    nodes_expanded = 0
    
    while heap:
        f, g, state, path = heapq.heappop(heap)
        state_str = str(state)
        
        if state == goal_state:
            return path + [state], nodes_expanded
            
        if state_str in visited:
            continue
            
        visited.add(state_str)
        nodes_expanded += 1
        
        # Find blank position (0)
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    x, y = i, j
        
        # Generate next states
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_state = [row[:] for row in state]
                new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
                new_g = g + 1
                new_f = new_g + h_misplaced(new_state)
                heapq.heappush(heap, (new_f, new_g, new_state, path + [state]))
    
    return None, nodes_expanded

In [ ]:
def a_star_manhattan(init_state, goal_state):
    def h_manhattan(state):
        distance = 0
        for i in range(3):
            for j in range(3):
                if state[i][j] != 0:
                    # Find the position of this number in goal state
                    for gi in range(3):
                        for gj in range(3):
                            if goal_state[gi][gj] == state[i][j]:
                                distance += abs(i - gi) + abs(j - gj)
                                break
        return distance
    
    heap = [(0 + h_manhattan(init_state), 0, init_state, [])]
    visited = set()
    nodes_expanded = 0
    
    while heap:
        f, g, state, path = heapq.heappop(heap)
        state_str = str(state)
        
        if state == goal_state:
            return path + [state], nodes_expanded
            
        if state_str in visited:
            continue
            
        visited.add(state_str)
        nodes_expanded += 1
        
        # Find blank position (0)
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    x, y = i, j
        
        # Generate next states
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_state = [row[:] for row in state]
                new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
                new_g = g + 1
                new_f = new_g + h_manhattan(new_state)
                heapq.heappush(heap, (new_f, new_g, new_state, path + [state]))
    
    return None, nodes_expanded

In [102]:
def print_result():
    # Goal state
    goal = [
        [0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]
    ]
    
    # Read puzzles
    puzzles = []
    filename = 'Input8PuzzleCases.txt'
    with open(filename, 'r') as f:
        for line in f:
            numbers = list(map(int, line.strip().split(', ')))
            puzzle = [numbers[i:i+3] for i in range(0, 9, 3)]
            puzzles.append(puzzle)
    
    # Statistics
    misplaced_steps = 0
    misplaced_time = 0
    manhattan_steps = 0
    manhattan_time = 0
    solved_count = 0
    
    for i, puzzle in enumerate(puzzles, 1):
        
        # Run A* with Misplaced Tile
        start_time = time.time()
        misplaced_sol, mt_nodes = a_star_misplaced(puzzle, goal)
        mt_time = time.time() - start_time
        
        # Run A* with Manhattan Distance
        start_time = time.time()
        manhattan_sol, mh_nodes = a_star_manhattan(puzzle, goal)
        mh_time = time.time() - start_time
        
        # Update statistics if solutions found
        if misplaced_sol and manhattan_sol:
            solved_count += 1
            mt_steps = len(misplaced_sol) - 1
            mh_steps = len(manhattan_sol) - 1
            
            misplaced_steps += mt_steps
            misplaced_time += mt_time
            manhattan_steps += mh_steps
            manhattan_time += mh_time
            
            # Print solution for first case only
            if i == 1:
                # print("\n" + "="*50)
                print_sol(misplaced_sol, "first", "A* Misplaced")
               
                # print("\n" + "="*50)
                print_sol(manhattan_sol, "first", "A* Manhattan")
                # print("="*50 + "\n")
    
    # Calculate and print statistics
    if solved_count > 0:
        print("\n" + " " * 16 + "Average_Steps    Average_Time")
        print(" " * 16 + "-" * 35)
        
        # Misplaced Tile results
        avg_mt_steps = misplaced_steps / solved_count
        avg_mt_time = misplaced_time / solved_count
        print(f" Misplaced{' ' * 8}{avg_mt_steps:.2f}{' ' * (16 - len(f'{avg_mt_steps:.2f}'))}{avg_mt_time:.4f}s")
        
        # Manhattan Distance results
        avg_mh_steps = manhattan_steps / solved_count
        avg_mh_time = manhattan_time / solved_count
        print(f" Manhattan{' ' * 7}{avg_mh_steps:.2f}{' ' * (16 - len(f'{avg_mh_steps:.2f}'))}{avg_mh_time:.4f}s")
        
    else:
        print("\nNo solvable cases found!")


You can insert as many cells as you want above
You are not Allowed to modify the code below this line.
## ===============================

In [103]:
#you need to implement print_result function to print out the result according to the required format
print_result()


Solution of the first Scenario using A* Misplaced:
8 7 5
4 1 2
3 0 6

to

8 7 5
4 0 2
3 1 6

to

8 7 5
4 2 0
3 1 6

to

8 7 0
4 2 5
3 1 6

to

8 0 7
4 2 5
3 1 6

to

8 2 7
4 0 5
3 1 6

to

8 2 7
4 1 5
3 0 6

to

8 2 7
4 1 5
3 6 0

to

8 2 7
4 1 0
3 6 5

to

8 2 0
4 1 7
3 6 5

to

8 0 2
4 1 7
3 6 5

to

0 8 2
4 1 7
3 6 5

to

4 8 2
0 1 7
3 6 5

to

4 8 2
1 0 7
3 6 5

to

4 0 2
1 8 7
3 6 5

to

0 4 2
1 8 7
3 6 5

to

1 4 2
0 8 7
3 6 5

to

1 4 2
3 8 7
0 6 5

to

1 4 2
3 8 7
6 0 5

to

1 4 2
3 0 7
6 8 5

to

1 4 2
3 7 0
6 8 5

to

1 4 2
3 7 5
6 8 0

to

1 4 2
3 7 5
6 0 8

to

1 4 2
3 0 5
6 7 8

to

1 0 2
3 4 5
6 7 8

to

0 1 2
3 4 5
6 7 8


Solution of the first Scenario using A* Manhattan:
8 7 5
4 1 2
3 0 6

to

8 7 5
4 0 2
3 1 6

to

8 7 5
4 2 0
3 1 6

to

8 7 0
4 2 5
3 1 6

to

8 0 7
4 2 5
3 1 6

to

8 2 7
4 0 5
3 1 6

to

8 2 7
4 1 5
3 0 6

to

8 2 7
4 1 5
3 6 0

to

8 2 7
4 1 0
3 6 5

to

8 2 0
4 1 7
3 6 5

to

8 0 2
4 1 7
3 6 5

to

0 8 2
4 1 7
3 6 5

to

4 8 2
0 1 7
3 6 5

to

4 8


# The output format should be as follows. You only need to give one sample solution as an example.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Solution of the first Scenario:
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### .
#### .
#### .
#### 0 1 2
#### 3 4 5
#### 6 7 8

                    Average_Steps    Average_Time      
 Miss Placed
 
 Manhattan Distance

